In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string, os 
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/lyrics-data.csv')
df.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


In [ ]:
# drop
df.drop(['ALink','SName','SLink'],axis=1,inplace=True)

In [ ]:
# value count
df['language'].value_counts()

en     191814
pt     157393
es       9917
rw       1679
it       1432
fr       1225
de        844
fi        145
sv        112
ro         97
no         89
is         86
tl         69
pl         47
gl         36
ga         32
tr         32
id         26
cy         23
su         19
af         19
sw         19
ko         17
nl         14
da         13
ca         13
et         13
ms          8
ja          7
st          7
ht          5
gd          4
ru          4
ar          4
eu          4
cs          3
ku          3
ny          3
mg          3
lg          2
jw          2
hu          2
vi          1
iw          1
sr          1
hmn         1
hr          1
fa          1
sq          1
zh          1
lv          1
sl          1
Name: language, dtype: int64

In [ ]:
#Exttracting just English language songs
df = df[df['language']=='en']

In [ ]:
df.columns

Index(['Lyric', 'language'], dtype='object')

In [ ]:
pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 499, in run
    confl

In [ ]:
from textblob import TextBlob

def sentiment_label(text):
    sentiment_score = TextBlob(text).sentiment.polarity
    if sentiment_score > 0:
        return 'happy'
    elif sentiment_score < 0:
        return 'sad'
    else:
        return 'neutral'

df['sentiment'] = df['Lyric'].apply(sentiment_label)

In [ ]:
df.head()

,Lyric,language,sentiment
69,I feel so unsure\nAs I take your hand and lead...,en,happy
86,"Don't let them fool, ya\nOr even try to school...",en,happy
88,"Baby, let's cruise, away from here\nDon't be c...",en,happy
111,"Know it sounds funny\nBut, I just can't stand ...",en,happy
140,You've got that look again\nThe one I hoped I ...,en,happy


In [ ]:
df['sentiment'].value_counts()

happy      134386
sad         54946
neutral      2482
Name: sentiment, dtype: int64

In [ ]:
rows_to_remove = df[df['sentiment'] == 'happy'].sample(n=79000)

In [ ]:
rows_to_remove['sentiment'].value_counts()

happy    79000
Name: sentiment, dtype: int64

In [ ]:
df_modified = df.drop(rows_to_remove.index)

In [ ]:
df_modified['sentiment'].value_counts()

happy      55386
sad        54946
neutral     2482
Name: sentiment, dtype: int64

In [ ]:
neutral_senti_rows_to_remove = df[df['sentiment'] == 'neutral']

In [ ]:
neutral_senti_rows_to_remove['sentiment'].value_counts()

neutral    2482
Name: sentiment, dtype: int64

In [ ]:
df_modified = df_modified.drop(neutral_senti_rows_to_remove.index)

In [ ]:
df_modified['sentiment'].value_counts()

happy    55386
sad      54946
Name: sentiment, dtype: int64

In [ ]:
df_modified.head()

,Lyric,language,sentiment
140,You've got that look again\nThe one I hoped I ...,en,happy
168,Everyone's feeling pretty\nIt's hotter than Ju...,en,happy
308,"When you're alone and you don't know how,\nTo ...",en,happy
968,If all that you have is money\n(You don't real...,en,happy
979,From the day we arrive on the planet\nAnd blin...,en,happy


In [ ]:
df_modified['lyrics_with_sentiment'] = df_modified['sentiment'] + ": " + df_modified['Lyric']

In [ ]:
df_modified.head()

,Lyric,language,sentiment,lyrics_with_sentiment
140,You've got that look again\nThe one I hoped I ...,en,happy,happy: You've got that look again\nThe one I h...
168,Everyone's feeling pretty\nIt's hotter than Ju...,en,happy,happy: Everyone's feeling pretty\nIt's hotter ...
308,"When you're alone and you don't know how,\nTo ...",en,happy,happy: When you're alone and you don't know ho...
968,If all that you have is money\n(You don't real...,en,happy,happy: If all that you have is money\n(You don...
979,From the day we arrive on the planet\nAnd blin...,en,happy,happy: From the day we arrive on the planet\nA...


In [ ]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.5 MB/s eta 0:00:00


In [ ]:
df_modified_truncated = df_modified.iloc[:10000]

In [ ]:
df_modified_truncated['lyrics_with_sentiment'].to_csv('lyrics_dataset_modified.txt', index=False, header=None)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

# Load pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Tokenize dataset and create TextDataset
def load_dataset(file_path, tokenizer):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )
    return dataset

train_dataset = load_dataset('lyrics_dataset_modified.txt', tokenizer)

# Data collator to create batches
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    save_steps=100,
    save_total_limit=2,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.518200
1000,3.456000
1500,3.439200
2000,3.378300
2500,3.288100
3000,3.310700
3500,3.311900
4000,3.297600
4500,3.278200
5000,3.268300


In [ ]:
trainer.save_model("./output")